In [1]:
import time
import pickle
from selenium import webdriver
from urllib.parse import urlparse
from bs4 import BeautifulSoup
from gsheet import authenticate
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import random

In [2]:
# Open Browser
browser = webdriver.Firefox()
# chrome = webdriver.Chrome()

In [25]:
# Constants
google = "https://google.com"
first_template_doc_url = "https://docs.google.com/document/d/1Fw-QKu1pSZdkIMPGuhPMTlgsoCum4lDttfNmvUOaeec/edit?usp=sharing"
second_template_doc_url = "https://docs.google.com/document/d/120haq0-0NTiDSZRRmrP8ebmUggM6-jcJMUCTzUsIwoU/edit?usp=sharing"
third_template_doc_url = "https://docs.google.com/document/d/1WCsAo5daOvqPbFfYNxnYmcPubzVYRKoCjWty2KhEwMA/edit"
outlook_office_url = "https://outlook.office.com/"
outlook_live_url = "https://outlook.live.com/"
fortest = "https://docs.google.com/spreadsheets/d/1UGhTjkslKEjoYA0mDe5l8GtS80hS39IsTe0YTE5s0a8/edit#gid=0"
gsheet_creds = "creds/s_a_creds.json"
tab_name = "Sheet1"
action = webdriver.ActionChains(browser)

In [10]:
# Functions
def open_new_tab():
    browser.execute_script("window.open()")
    
    
def switch_to_tab(number):
    browser.switch_to.window(browser.window_handles[number])


# Load Old Cookies (If exists)
def load_cookies(driver, location, url=None):
    cookies = pickle.load(open(location, "rb"))
    driver.delete_all_cookies()
    url = "https://www.google.com" if url is None else url
    driver.get(url)
    for cookie in cookies:
        driver.add_cookie(cookie)


# Credentials for used account
def get_email_creds(file_name):
    fl = open(file_name).readlines()
    email = fl[0]
    password = fl[1]
    return [email, password]


# Go to google docs url and copy the template
# def grab_template(template):
#     browser.get(template)
#     time.sleep(5)
#     action.key_down(Keys.CONTROL).send_keys("a").key_up(Keys.CONTROL).perform()
#     time.sleep(1)
#     action.key_down(Keys.CONTROL).send_keys("c").key_up(Keys.CONTROL).perform()

def grab_template(url):
    open_new_tab()
    switch_to_tab(1)
    browser.get(url)
    time.sleep(5)
    action.key_down(Keys.CONTROL).send_keys("a").key_up(Keys.CONTROL).perform()
    time.sleep(1)
    action.key_down(Keys.CONTROL).send_keys("c").key_up(Keys.CONTROL).perform()
    switch_to_tab(0)

    
# Login to Normal Outlook Live Account
def login_to_outlook_live(email, password):
    browser.get("https://login.live.com")
    time.sleep(3)
    mail_id = browser.find_element_by_name("loginfmt")
    mail_id.send_keys(email)
    mail_id.send_keys(Keys.RETURN)
    time.sleep(2)
    passwd = browser.find_element_by_name("passwd")
    passwd.send_keys(password)
    passwd.submit()
    time.sleep(3)
    browser.get("https://outlook.live.com")


# Login to Professional Outlook Office account
def login_to_outlook_office(email, password):
    browser.get("https://outlook.office.com")
    time.sleep(3)
    mail_id = browser.find_element_by_name("loginfmt")
    mail_id.send_keys(email)
    mail_id.send_keys(Keys.RETURN)
    time.sleep(2)
    passwd = browser.find_element_by_name("passwd")
    passwd.send_keys(password)
    passwd.submit()
    time.sleep(3)
    browser.get("https://outlook.office.com")
    

# Connect To The Google Sheets API
def connect_to_gsheet(credentials, url, tab_name):
    client = authenticate(credentials)
    workbook = client.open_by_url(url)
    selected_tab = workbook.worksheet(tab_name)
    return selected_tab
    
    
# Send An Email
def send_email_from_sarah(email, first_name, company_name, duration):
    new_msg = browser.find_elements_by_class_name("_33rLSYbzxvhXjgYTwfjWQI")
    new_msg[0].click()
    time.sleep(duration)
    to = browser.find_elements_by_class_name("pickerInput_cf204fb8")
    to[0].send_keys(email)
    time.sleep(duration)
    to[0].send_keys(Keys.ENTER)
    time.sleep(2)
    subject = browser.find_elements_by_xpath("//input[@placeholder='Add a subject']")
    subject[0].click()
    subject[0].send_keys(f"Private, online data science training exclusively for {company_name}")
    time.sleep(duration)
    msg_body = browser.find_elements_by_xpath("//div[@aria-label='Message body']")
    msg_body[0].click()
    msg_body[0].send_keys(f"Hi {first_name},\n\n")
    time.sleep(2)
    msg_body[0].send_keys(Keys.CONTROL + "v")
    time.sleep(duration)
    msg_body[0].send_keys(Keys.BACKSPACE)
    time.sleep(2)
#     confirmation = input(f"Before sending please check that,\nThe Person's name is {first_name},\nHis mail address is {email}\nAnd he works in {company_name}\nPress ENTER to SEND the mail to {first_name}")
#     if confirmation.lower() == "":
    msg_body[0].send_keys(Keys.CONTROL + Keys.ENTER)
    print(f"Email sent to {first_name}\n")
#     else:
#         print(f"The mail is saved to the Draft Folder.\n")




In [4]:
# use outlook live account
# email_creds = get_email_creds("creds/outlook_neela.txt")
# login_to_outlook_live(email_creds[0], email_creds[1])
# time.sleep(2)
# pickle.dump(browser.get_cookies(), open("cookies/outlook_neela.txt", "wb"))

# Use Outlook Office Account
# try:
#     load_cookies(browser, "cookies/outlook_sarah.txt", url=outlook_office_url)
# except:
email_creds = get_email_creds("creds/outlook_sarah.txt")
login_to_outlook_office(email_creds[0], email_creds[1])
time.sleep(2)
# pickle.dump(browser.get_cookies(), open("cookies/outlook_sarah.txt", "wb"))


selected_tab = connect_to_gsheet(gsheet_creds, fortest, tab_name)


In [5]:
grab_template(first_template_doc_url)
browser.get(outlook_office_url)

In [7]:
# Select rows and send mail to them
print("Google Sheet Row Data:")
starting_row = int(input("Enter the Starting Row Number: "))
finishing_row = int(input("Enter the Finishing Row Number: "))
companies = selected_tab.get(f"A{starting_row}:A{finishing_row}")
first_names = selected_tab.get(f"E{starting_row}:E{finishing_row}")
emails = selected_tab.get(f"H{starting_row}:H{finishing_row}")
total_rows = finishing_row - starting_row + 1
print(f"You have selected {total_rows} rows.\n")
# Loop
for i in range(total_rows):
    email = emails[i][0]
    company_name = companies[i][0]
    first_name = first_names[i][0]
    x = random.randint(5, 12)
    send_email_from_sarah(email, first_name, company_name, x)
    if i == total_rows - 1:
        print(f"\n{total_rows} emails sent.")

Google Sheet Row Data:
Enter the Starting Row Number: 1301
Enter the Finishing Row Number: 1340
You have selected 40 rows.

Email sent to Vasanth

Email sent to Miroslav

Email sent to Alexandru

Email sent to Ray

Email sent to Alexandra

Email sent to Kristen

Email sent to Brian

Email sent to Michael

Email sent to Michael

Email sent to Tim

Email sent to Benjamin

Email sent to Dago

Email sent to Frank

Email sent to Todd

Email sent to Savitha

Email sent to Christopher

Email sent to Matthew

Email sent to Adam

Email sent to Phillip

Email sent to Will

Email sent to Shani

Email sent to Danny

Email sent to Katie

Email sent to Sheila

Email sent to David

Email sent to John

Email sent to David

Email sent to Jeff

Email sent to Kasia

Email sent to Raquel

Email sent to Ryan

Email sent to Christopher

Email sent to Caroline

Email sent to Daniel

Email sent to Dan

Email sent to Jacqueline

Email sent to Joseph

Email sent to Lisa

Email sent to Sanjiv

Email sent to John

In [8]:
## 2nd Follow-ups

In [11]:
grab_template(second_template_doc_url)

In [23]:
# Select rows and send mail to them
print("Google Sheet Row Data:")
starting_row = int(input("Enter the Starting Row Number: "))
finishing_row = int(input("Enter the Finishing Row Number: "))
first_names = selected_tab.get(f"E{starting_row}:E{finishing_row}")
emails = selected_tab.get(f"H{starting_row}:H{finishing_row}")
total_rows = finishing_row - starting_row + 1
print(f"You have selected {total_rows} rows.\n")
# Loop
for i in range(total_rows):
    email = emails[i][0]
    first_name = first_names[i][0]
    duration = random.randint(10,25)
    print(f"Sending followup to {email}")
    try:
        search_bar = browser.find_element_by_xpath("//*[@aria-label='Search']")
        search_bar.click()
        search_bar.clear()
        search_bar.send_keys(email)
        search_bar.send_keys(Keys.ENTER)
        time.sleep(4)
        search_result_list = browser.find_element_by_xpath("//*[@aria-label='Message list No conversations selected']")
        search_result_list.send_keys(Keys.DOWN)
        time.sleep(2)
        content_pane = browser.find_element_by_xpath("//*[@aria-label='Content pane']")
        delivery_check = content_pane.text
        failing_texts = [
            "delivery has failed",
            "couldn't be delivered",
            "permanent fatal errors",
            "address not found",
            "your message wasn't delivered"
        ]
        is_failed = False
        for text in failing_texts:
            if text in delivery_check.lower():
                is_failed = True

        if not is_failed:
            reply = browser.find_element_by_name("Reply")
            reply.click()
        #     switch_to_tab(1)
        #     action.key_down(Keys.CONTROL).send_keys("a").key_up(Keys.CONTROL).perform()
        #     time.sleep(1)
        #     action.key_down(Keys.CONTROL).send_keys("c").key_up(Keys.CONTROL).perform()
        #     time.sleep(1)
        #     switch_to_tab(0)
            time.sleep(2)
            msg_body = browser.find_element_by_xpath("//*[@aria-label='Message body']")
            msg_body.send_keys(f"Hi {first_name}-\n\n")
            time.sleep(3)
            msg_body.send_keys(Keys.CONTROL + "v")
            time.sleep(2)
            msg_body.send_keys(Keys.BACKSPACE)
            time.sleep(duration)
#             x = input("please make sure: ")
#             if x == "":
            msg_body.send_keys(Keys.CONTROL + Keys.ENTER)
            print(f"Successful: {email}")
        else:
            print(f"***Failed Mail: {email}")
    except:
        print(f"***Error Faced: {email}")
    time.sleep(2)
    if i == total_rows - 1:
        print(f"\n{total_rows} followups sent.")

Google Sheet Row Data:
Enter the Starting Row Number: 569
Enter the Finishing Row Number: 610
You have selected 42 rows.

Sending followup to ptjones@stifel.com
Successful: ptjones@stifel.com
Sending followup to schultzea@stifel.com
Successful: schultzea@stifel.com
Sending followup to stacya@stifel.com
Successful: stacya@stifel.com
Sending followup to troderick@stifel.com
Successful: troderick@stifel.com
Sending followup to aorzell@stubhub.com
Successful: aorzell@stubhub.com
Sending followup to bengel@stubhub.com
Successful: bengel@stubhub.com
Sending followup to briprice@stubhub.com
Successful: briprice@stubhub.com
Sending followup to carsong@stubhub.com
Successful: carsong@stubhub.com
Sending followup to correed@stubhub.com
Successful: correed@stubhub.com
Sending followup to dmanandhar@stubhub.com
Successful: dmanandhar@stubhub.com
Sending followup to droutela@stubhub.com
Successful: droutela@stubhub.com
Sending followup to dsayman@stubhub.com
Successful: dsayman@stubhub.com
Sending 

In [24]:
## 3rd Follow-ups

In [26]:
grab_template(third_template_doc_url)

In [29]:
# Select rows and send mail to them
print("Google Sheet Row Data:")
starting_row = int(input("Enter the Starting Row Number: "))
finishing_row = int(input("Enter the Finishing Row Number: "))
first_names = selected_tab.get(f"E{starting_row}:E{finishing_row}")
emails = selected_tab.get(f"H{starting_row}:H{finishing_row}")
total_rows = finishing_row - starting_row + 1
print(f"You have selected {total_rows} rows.\n")
# Loop
for i in range(total_rows):
    email = emails[i][0]
    first_name = first_names[i][0]
    duration = random.randint(10,25)
    print(f"Sending followup to {email}")
    try:
        search_bar = browser.find_element_by_xpath("//*[@aria-label='Search']")
        search_bar.click()
        search_bar.clear()
        search_bar.send_keys(email)
        search_bar.send_keys(Keys.ENTER)
        time.sleep(4)
        search_result_list = browser.find_element_by_xpath("//*[@aria-label='Message list No conversations selected']")
        search_result_list.send_keys(Keys.DOWN)
        time.sleep(2)
        content_pane = browser.find_element_by_xpath("//*[@aria-label='Content pane']")
        delivery_check = content_pane.text
        failing_texts = [
            "delivery has failed",
            "couldn't be delivered",
            "permanent fatal errors",
            "address not found",
            "your message wasn't delivered"
        ]
        is_failed = False
        for text in failing_texts:
            if text in delivery_check.lower():
                is_failed = True

        if not is_failed:
            reply = browser.find_element_by_name("Reply")
            reply.click()
        #     switch_to_tab(1)
        #     action.key_down(Keys.CONTROL).send_keys("a").key_up(Keys.CONTROL).perform()
        #     time.sleep(1)
        #     action.key_down(Keys.CONTROL).send_keys("c").key_up(Keys.CONTROL).perform()
        #     time.sleep(1)
        #     switch_to_tab(0)
            time.sleep(2)
            msg_body = browser.find_element_by_xpath("//*[@aria-label='Message body']")
            msg_body.send_keys(f"Hi {first_name}-\n\n")
            time.sleep(3)
            msg_body.send_keys(Keys.CONTROL + "v")
            time.sleep(2)
            msg_body.send_keys(Keys.BACKSPACE)
            time.sleep(duration)
#             x = input("please make sure: ")
#             if x == "":
            msg_body.send_keys(Keys.CONTROL + Keys.ENTER)
            print(f"Successful: {email}")
        else:
            print(f"***Failed Mail: {email}")
    except:
        print(f"***Error Faced: {email}")
    time.sleep(2)
    if i == total_rows - 1:
        print(f"\n{total_rows} followups sent.")

Google Sheet Row Data:


KeyboardInterrupt: Interrupted by user